# 10 Image Data Augmentation

*IMPORTANT*

If you're experiencing issues with this notebook, please update your ```cuDNN``` library:

```
conda activate tf
pip install nvidia-cudnn-cu11==8.9.0.131
```

Then, import a familiar set of libraries:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image as image
import tensorflow as tf
from tensorflow import keras

The final problem on the last sheet gave you a sneak-peak at one of the most important topics in data preprocessing - data augmentation. Tensorflow allows us to augment our training data in various fashions on the fly:

- Rotations
- Translations (vertical/horizontal)
- Crops/Zoom
- Flips (vertical/horizontal)
- ...

For the full list, check https://keras.io/api/preprocessing/image/. In Tensorflow, image data augmentation is carried out by creating a so-called data generator which is then called by the Tensorflow ``fit`` method to retrieve a batch of augmentated data which are then fed into the network. Here, we will create a generator for the MNIST dataset of handwritten digits:

In [ ]:
mnist = keras.datasets.mnist

(trainImages, trainLabels), (testImages, testLabels) = mnist.load_data()

print(trainImages.shape, testImages.shape)

Let's create a simple data generator that returns random digits rotated by up to 10 deg, where empty pixels are filled with 0:

In [ ]:
imageGenerator = keras.preprocessing.image.ImageDataGenerator(
    fill_mode='constant',
    cval = 0.,
    rotation_range=10,
#    width_shift_range=.10,
#    height_shift_range=.10,
    horizontal_flip=False,
    vertical_flip=False
)

The generator expects a image data array with four dimensions to deal with color images. Therefore, we need to add a dummy axis:

In [ ]:
trainImages = np.expand_dims(trainImages, axis=3)
print(trainImages.shape)

We can now create and compile a model:

In [ ]:
readNet = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28, 1)),
    keras.layers.Dense(200, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

readNet.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

To train the model, we pass the ```imageGenerator.flow``` with the training data and labels as input for the ```fit``` method:

In [ ]:
batch = 200

history = readNet.fit(imageGenerator.flow(trainImages, trainLabels, batch_size=batch, shuffle=False),validation_data=(testImages, testLabels), steps_per_epoch=len(trainImages) / batch, epochs=200)

``batch`` defines the number of images that are generated every time we call the image generator. If you're using ``tensorflow-gpu`` and run out of memory (``resource exhaustion error``), try to reduce the batch size. On a desktop CPU, training the network takes forever ...